# 🚀 Exemplos Avançados e Aplicações

**Tutorial de Espaço Latente - Notebook 6**

## 🎯 Objetivos
- Técnicas avançadas de treinamento
- Aplicações práticas de VAEs
- Dicas de debugging e otimização
- Extensões e próximos passos
- Integrando tudo que aprendemos

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from src.models.vae import VAE
from src.models.beta_vae import BetaVAE, AnnealedBetaVAE
from src.utils.data_loader import load_mnist
from src.utils.training import train_vae, EarlyStopping
from src.utils.visualization import *
from src.experiments import LatentExplorer, BetaVAEComparison

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

## 🎯 Aplicação 1: Detecção de Anomalias

VAEs podem detectar imagens "anômalas" através do erro de reconstrução.

In [ ]:
# Carrega dados
train_loader, val_loader, test_loader = load_mnist(batch_size=128)

# Treina VAE
vae = VAE(input_dim=784, latent_dim=10, hidden_dims=[512, 256])

history = train_vae(
    model=vae,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=20,
    device=DEVICE,
    verbose=False
)

print("✅ VAE trained!")

In [ ]:
# Calcula reconstruction error para todo test set
vae.eval()
reconstruction_errors = []
all_labels = []

with torch.no_grad():
    for data, labels in test_loader:
        data = data.view(-1, 784).to(DEVICE)
        recon, mu, logvar, z = vae(data)
        
        # Erro por amostra
        errors = ((data - recon) ** 2).sum(dim=1).cpu().numpy()
        reconstruction_errors.extend(errors)
        all_labels.extend(labels.numpy())

reconstruction_errors = np.array(reconstruction_errors)
all_labels = np.array(all_labels)

# Visualiza distribuição de erros por dígito
fig, ax = plt.subplots(figsize=(12, 6))

for digit in range(10):
    mask = all_labels == digit
    errors_digit = reconstruction_errors[mask]
    ax.hist(errors_digit, bins=50, alpha=0.5, label=f'Digit {digit}')

ax.set_xlabel('Reconstruction Error', fontweight='bold')
ax.set_ylabel('Count', fontweight='bold')
ax.set_title('Reconstruction Error Distribution by Digit', fontweight='bold', fontsize=14)
ax.legend(ncol=5)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Encontra anomalias (top 1% maior erro)
threshold = np.percentile(reconstruction_errors, 99)
anomaly_indices = np.where(reconstruction_errors > threshold)[0]

print(f"\n🔍 Anomaly Detection:")
print(f"  Threshold (99th percentile): {threshold:.2f}")
print(f"  Number of anomalies: {len(anomaly_indices)}")
print(f"  Anomaly rate: {100*len(anomaly_indices)/len(reconstruction_errors):.2f}%")

In [ ]:
# Visualiza anomalias
data_all = torch.cat([d for d, _ in test_loader]).view(-1, 784)

fig, axes = plt.subplots(2, 10, figsize=(15, 3))

for i in range(10):
    idx = anomaly_indices[i]
    img = data_all[idx].view(28, 28)
    error = reconstruction_errors[idx]
    label = all_labels[idx]
    
    axes[0, i].imshow(img, cmap='gray')
    axes[0, i].set_title(f'Label: {label}\nError: {error:.1f}', fontsize=9)
    axes[0, i].axis('off')
    
    # Reconstrução
    with torch.no_grad():
        recon, _, _, _ = vae(data_all[idx:idx+1].to(DEVICE))
        recon_img = recon.view(28, 28).cpu()
    
    axes[1, i].imshow(recon_img, cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[0, i].set_ylabel('Original', fontweight='bold', fontsize=11)
        axes[1, i].set_ylabel('Recon', fontweight='bold', fontsize=11)

plt.suptitle('Top 10 Anomalies (Highest Reconstruction Error)', fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Imagens com alto erro podem ser:\n  - Mal escritas\n  - Ambíguas\n  - Raras no dataset")

## 🎨 Aplicação 2: Data Augmentation

Use VAE para gerar variações de dados existentes.

In [ ]:
def augment_data(vae, image, n_augmentations=5, noise_scale=0.3):
    """Gera variações de uma imagem adicionando ruído no espaço latente."""
    vae.eval()
    
    with torch.no_grad():
        # Encode
        img_tensor = image.view(1, -1).to(DEVICE)
        mu, logvar = vae.encode(img_tensor)
        
        augmented = []
        for _ in range(n_augmentations):
            # Adiciona ruído gaussiano no espaço latente
            noise = torch.randn_like(mu) * noise_scale
            z_augmented = mu + noise
            
            # Decode
            aug_img = vae.decode(z_augmented)
            augmented.append(aug_img.view(28, 28).cpu())
    
    return augmented

# Pega uma imagem
sample_img, sample_label = next(iter(test_loader))
original = sample_img[0]

# Gera augmentações
augmented_images = augment_data(vae, original, n_augmentations=9)

# Visualiza
fig, axes = plt.subplots(2, 5, figsize=(12, 5))

axes[0, 0].imshow(original.squeeze(), cmap='gray')
axes[0, 0].set_title('Original', fontweight='bold')
axes[0, 0].axis('off')

for i in range(9):
    row = (i+1) // 5
    col = (i+1) % 5
    axes[row, col].imshow(augmented_images[i], cmap='gray')
    axes[row, col].set_title(f'Aug {i+1}', fontsize=10)
    axes[row, col].axis('off')

plt.suptitle(f'Data Augmentation via Latent Space Perturbation (Label: {sample_label[0]})',
            fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

## 🧪 Aplicação 3: Latent Space Interpolation Path Finding

Encontra o caminho "mais suave" entre duas imagens.

In [ ]:
# Interpolação esférica (SLERP) vs linear
def slerp(z1, z2, t):
    """Spherical Linear Interpolation."""
    omega = torch.acos((z1 * z2).sum() / (torch.norm(z1) * torch.norm(z2)))
    so = torch.sin(omega)
    return torch.sin((1-t)*omega) / so * z1 + torch.sin(t*omega) / so * z2

# Pega duas imagens
img1, img2 = sample_img[0], sample_img[5]

# Encode
vae.eval()
with torch.no_grad():
    mu1, _ = vae.encode(img1.view(1, -1).to(DEVICE))
    mu2, _ = vae.encode(img2.view(1, -1).to(DEVICE))

# Interpolações
n_steps = 10
alphas = torch.linspace(0, 1, n_steps)

linear_interp = []
slerp_interp = []

with torch.no_grad():
    for alpha in alphas:
        # Linear
        z_linear = (1 - alpha) * mu1 + alpha * mu2
        img_linear = vae.decode(z_linear).view(28, 28).cpu()
        linear_interp.append(img_linear)
        
        # SLERP
        z_slerp = slerp(mu1[0], mu2[0], alpha).unsqueeze(0)
        img_slerp = vae.decode(z_slerp).view(28, 28).cpu()
        slerp_interp.append(img_slerp)

# Visualiza
fig, axes = plt.subplots(2, n_steps, figsize=(15, 3))

for i in range(n_steps):
    axes[0, i].imshow(linear_interp[i], cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_title('Linear', fontweight='bold', loc='left')
    
    axes[1, i].imshow(slerp_interp[i], cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_title('SLERP', fontweight='bold', loc='left')

plt.suptitle('Linear vs Spherical Interpolation', fontweight='bold')
plt.tight_layout()
plt.show()

## 🔧 Técnica Avançada: Scheduled Sampling

Durante treinamento, gradualmente usa próprias predições ao invés de dados reais.

In [ ]:
# Exemplo conceitual (não implementado completamente)
print("📚 Scheduled Sampling Concept:")
print("\nEpoch 1-5: Use 100% dados reais")
print("Epoch 6-10: Use 80% dados reais, 20% reconstruções")
print("Epoch 11-15: Use 60% dados reais, 40% reconstruções")
print("...")
print("\n💡 Força o modelo a confiar nas próprias predições!")

## 🐛 Debugging: Visualizando Gradientes

In [ ]:
# Função para plotar magnitude dos gradientes
def plot_grad_flow(named_parameters):
    """Plota magnitude dos gradientes para debug."""
    ave_grads = []
    max_grads = []
    layers = []
    
    for n, p in named_parameters:
        if p.requires_grad and p.grad is not None:
            layers.append(n)
            ave_grads.append(p.grad.abs().mean().item())
            max_grads.append(p.grad.abs().max().item())
    
    plt.figure(figsize=(12, 6))
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.5, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.5, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k")
    plt.xticks(range(0, len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom=-0.001, top=max(max_grads)*1.1)
    plt.xlabel("Layers")
    plt.ylabel("Gradient Magnitude")
    plt.title("Gradient Flow")
    plt.grid(True, alpha=0.3)
    plt.legend(["max-gradient", "mean-gradient"])
    plt.tight_layout()
    plt.show()

# Exemplo de uso
data, _ = next(iter(train_loader))
data = data.view(-1, 784).to(DEVICE)

vae.train()
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

# Forward + backward
from src.models.vae import vae_loss
recon, mu, logvar, z = vae(data)
loss_dict = vae_loss(recon, data, mu, logvar)
loss = loss_dict['total']

optimizer.zero_grad()
loss.backward()

# Plota gradientes
plot_grad_flow(vae.named_parameters())

## 📊 Pipeline Completo: Do Zero até Aplicação

In [ ]:
print("🚀 Complete VAE Pipeline:\n")
print("1. Load Data")
print("   → load_mnist(batch_size=128)")
print("\n2. Create Model")
print("   → VAE(latent_dim=10, hidden_dims=[512, 256])")
print("\n3. Train")
print("   → train_vae(model, train_loader, val_loader, epochs=30)")
print("   → Use early_stopping_patience=5")
print("   → Save best model")
print("\n4. Evaluate")
print("   → visualize_latent_space()")
print("   → plot_reconstructions()")
print("   → plot_vae_results()")
print("\n5. Apply")
print("   → Anomaly detection")
print("   → Data augmentation")
print("   → Generation")
print("   → Latent space manipulation")
print("\n6. Experiment")
print("   → Try different betas")
print("   → Different latent dimensions")
print("   → Different architectures")
print("\n✅ You now have all the tools!")

## 📚 Recursos Adicionais e Próximos Passos

### 📖 Leitura Recomendada
1. **Original VAE Paper**: "Auto-Encoding Variational Bayes" (Kingma & Welling, 2013)
2. **Beta-VAE**: "β-VAE: Learning Basic Visual Concepts" (Higgins et al., 2017)
3. **Understanding VAEs**: Tutorial by Carl Doersch (2016)

### 🚀 Extensões Possíveis
1. **Conditional VAE (CVAE)**: Condicionar geração em labels
2. **Hierarchical VAE**: Múltiplos níveis latentes
3. **VQ-VAE**: Quantização vetorial
4. **Adversarial VAE**: Combinar com GANs

### 🛠️ Projetos Práticos
1. Aplicar em outros datasets (CIFAR-10, CelebA)
2. Style transfer no espaço latente
3. Compressão de imagens
4. Detecção de fraudes
5. Recomendação de produtos

### 🎯 Desafios
1. Treinar VAE com latent_dim=50
2. Implementar CVAE
3. Criar interface web para exploração
4. Aplicar em dataset próprio
5. Comparar VAE com Autoencoder em tarefa específica

## 🎓 Conclusão do Tutorial

### O que você aprendeu:

✅ **Notebook 1**: Conceitos fundamentais de espaço latente  
✅ **Notebook 2**: Implementação e treinamento de Autoencoder  
✅ **Notebook 3**: VAE e espaço latente probabilístico  
✅ **Notebook 4**: Beta-VAE e disentanglement  
✅ **Notebook 5**: Exploração interativa  
✅ **Notebook 6**: Aplicações avançadas  

### Habilidades Adquiridas:
- Treinar e avaliar VAEs
- Visualizar espaços latentes
- Gerar novas amostras
- Detectar anomalias
- Manipular representações latentes
- Debugar modelos generativos

### 🌟 Você está pronto para:
- Aplicar VAEs em problemas reais
- Explorar variantes mais avançadas
- Contribuir para projetos de ML
- Ensinar outros sobre espaços latentes!

---

## 🙏 Obrigado por completar este tutorial!

**Professora Itamar - UTFPR**

Continue explorando e experimentando! 🚀